# Tarea 1 AS4501-1

Profesor de catedra: Francisco Förster

Autor: Álvaro Cáceres

In [ ]:
## Importing libraries
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import corner
import astropy.units as u 
import astropy.cosmology
import dill
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from mpl_toolkits.mplot3d import Axes3D

## Problem 1
proposito blablabla

The file `fundamental-plane.csv` precisely contains data about these parameters originally compiled by Djorgovski & Davis in 1987. Moving forward in this problem we will be working with this data.

In [14]:
# We load and see the dataset with pandas
data = pd.read_csv("fundamental_plane.csv")
data

,ID,log_re_arcsec,err_log_re_arcsec,log_re_pc,err_log_re_pc,m,err_m,M,err_M,mu,err_mu,ellip,err_elip,log_sigma,err_log_sigma
0,NGC-57,1.29,0.02,3.70,0.02,11.86,0.44,-21.76,0.44,19.28,0.47,0.17,0.02,2.502,0.025
1,NGC-97,1.18,0.01,3.54,0.01,12.44,0.16,-20.92,0.16,19.55,0.17,0.06,0.02,2.086,0.117
2,NGC-194,1.40,0.01,3.79,0.01,11.94,0.06,-21.55,0.06,20.14,0.08,0.11,0.02,2.332,0.051
3,NGC-221,1.65,0.01,2.16,0.05,7.89,0.21,-16.21,0.31,17.27,0.22,0.17,0.02,1.898,0.055
4,NGC-410,1.62,0.01,4.03,0.01,11.26,0.30,-22.36,0.30,20.37,0.32,0.26,0.02,2.458,0.023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,NGC-7785,1.55,0.02,3.80,0.02,11.23,0.17,-21.57,0.17,19.65,0.21,0.44,0.01,2.382,0.045
102,IC-179,1.15,0.01,3.47,0.02,12.45,0.14,-20.73,0.14,19.28,0.17,0.16,0.02,2.447,0.030
103,IC-962,0.80,0.04,3.31,0.04,13.24,0.22,-20.86,0.22,18.31,0.28,0.10,0.01,2.185,0.086
104,IC-1211,1.10,0.02,3.56,0.02,12.75,0.13,-21.11,0.13,19.33,0.17,0.13,0.01,2.212,0.123


In [44]:
# We define the X (independent) and Y (dependent) variables for further use in the linear regression

fplane_X = data[['log_sigma', 'mu']]
fplane_Y = data[['log_re_pc']]

# We split the data into training/testing sets

fplane_X_train, fplane_X_test, fplane_Y_train, fplane_Y_test = train_test_split(fplane_X, fplane_Y, test_size=0.3, random_state=27)

# We create prediction using the testing set

fplane_Y_pred = regr.predict(fplane_X_test)

In [54]:
# We create our first (simple) regression

regr = LinearRegression()
regr.fit (fplane_X_train, fplane_Y_train)
regr.pred = regr.predict(fplane_X_test)

# We print our coefficients, MSE and variance

print('Coefficients: \n', regr.coef_, regr.intercept_)
print('Mean squared error (MSE): %.2f' % mean_squared_error(fplane_Y_test, fplane_Y_pred))
print('Variance score (R2): %.2f' % r2_score(fplane_Y_test, fplane_Y_pred))

Coefficients: 
 [[1.14242356 0.31094493]] [-5.15997076]
Mean squared error (MSE): 0.03
Variance score (R2): 0.74


AAAAAAAAAAAAAAA